In [1]:
# STDP unsupervised SNN learning model
# Pruned
# Digit recognition (MNIST)

import numpy as np 
import struct
import matplotlib.pyplot as plt

from brian2 import *

%matplotlib inline

INFO       Cache size for target "cython": 1971 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\Lei\.cython\brian_extensions" directory. [brian2]


In [15]:
# read in test data

batch_tot = 1

data_test = []

n_input = 0
n_sample = 0

for batch_index in range(batch_tot):
    with open("./dataset/MNIST/pst_test_{0}.txt".format(batch_index), "r") as f:
        n_inputNeuron, n_newSample = [int(x) for x in next(f).split()]
        if (batch_index == 0):
            n_input = n_inputNeuron
        n_sample = n_sample + n_newSample
        for line in f:
            data_test.append([float(x) for x in line.split()])


# read in MNIST labels

label_test = []
n_sample = 10000

with open("./dataset/MNIST/test_label.txt", "r") as f:
    for line in f:
        label_test.append(int(line))

In [16]:
# read in synapse weight
synapse_weight = []
with open("./input_synapse_weight_0-3-20.txt", "r") as f:
    n_synapse = [int(x) for x in next(f).split()]
    for line in f:
        newdata = [float(x) for x in line.split()]
        newdata[:2] = [int(x) for x in newdata[:2]]
        synapse_weight.append(newdata)

In [85]:
# Set up parameter for modle training
# Network Construction
start_scope()

prefs.codegen.max_cache_dir_size = 0

# Model parameters ===========================================================
# size of first layer
n_input = 784

# synapse weight threshold for pruning
w_th = 0.5

# neuron parameters *******************************************************
# input neuron  -----------------------------------------------------------
# input spike rate multiplier
rate_multiplier = 10*Hz

# excitatory neuron parameters  ---------------------------------------------
# number of excitatory neurons
n_exc = 100

# membrane time constant
tau_exc = 50*ms

# initial threshold
vt_exc_0 = 0.02

# initial rest threshold
vt_exc_rest_0 = 0.02

# threshold increment ratio
vt_exc_inc = 0.05

# max threshold
vt_exc_max = 1

# threshold decrese time constant
# (was 2000*ms)
vt_exc_tau = 1000*ms

# neuron dynamics
eqs_exc = '''
dv/dt = -v/tau_exc : 1
vt_exc : 1
n_fire : 1
'''

# inhibitory neuron parameters ---------------------------------------------
# number of inhibitory neurons
n_inh = 100

# membrane time constant
tau_inh = 50 * ms

# threshold
vt_inh = 0.2

# neuron dynamics
eqs_inh = '''
dv/dt = -v/tau_inh : 1
'''

# synapse parameters  ***********************************************

# input synapses ---------------------------------------------------
# neuron potential scaler
scaler = 0.1 / 2500

# synapse dynamics
synapse_dynamic = '''
w : 1
'''

synapse_pre_action = '''
v_post += w * scaler
'''

# v_post = clip(v_post + w, 0, 1)v_post = clip(v_post + w, 0, 1)

# synapse_post_action = '''
# delta_w = eta * ((exp((pre_last - t) / tauw) - offset))
# delta_w = delta_w * (wmax - w) * int(delta_w > 0) + delta_w * (w - wmin) * int(delta_w < 0)
# w = clip(w + delta_w, 0, 1)
# '''

# synapse_post_action = '''
# new_eta = eta
# delta_w = eta * ((exp((pre_last - t) / tauw) - offset)) * (wmax - w)
# w = clip(w + delta_w, 0, 1)
# '''

# excitatory synapses ----------------------------------------------------
# synapse weight
exc_w_0 = 1

# synapse dynamics
exc_dynamic = '''
exc_w : 1
'''

# synapse on pre action
exc_pre_action = '''
v_post += exc_w
'''

# inhibitory synapses ----------------------------------------------------
# synapse weight
inh_w_0 = 0.1

# synapse dynamics
inh_dynamic = '''
inh_w : 1
'''

# synapse on pre action
inh_pre_action = '''
v_post = v_post * inh_w_0
'''

# Set up neurons ==========================================================
# set up input layer ******************************************************

input_layer = PoissonGroup(n_input, data_test[0]*rate_multiplier)


# set up excitatory layer *************************************************
exc_layer = NeuronGroup(
    n_exc, 
    eqs_exc, 
    threshold="v>vt_exc", 
    refractory="5*ms", 
    reset="v=0", 
    method="exact",
    events={"fire_record" : "v>vt_exc"})

exc_layer.vt_exc = vt_exc_0
exc_layer.n_fire = 0

# neuron action on spike firing event
exc_layer.run_on_event("fire_record", '''n_fire += 1''')

# set up inhibitory layer **************************************************

inh_layer = NeuronGroup(
    n_inh,
    eqs_inh,
    threshold = "v>vt_inh",
    refractory = "3*ms",
    reset="v=0",
    method="exact"
)


# Set up synapses =============================================================
# Synapses from input layer to excitatory layer *******************************
input_s = Synapses(
    input_layer, 
    exc_layer, 
    synapse_dynamic,
    method = "exact",
    on_pre = synapse_pre_action)

input_s.connect()

# Initialize / Randomize input synapses
# for src_index in range(n_input):
#     for dst_index in range(exc_layer.N):
#         input_s.w[src_index, dst_index] = eta * uniform(0.1, 0.9)
for synapse_info in synapse_weight:
    input_s.w[synapse_info[0], synapse_info[1]] = synapse_info[2]

# Synapses from exc to inh ****************************************************
exc_s = Synapses(
    exc_layer,
    inh_layer,
    exc_dynamic,
    on_pre = exc_pre_action
)

exc_s.connect(condition = "i == j")

# initialize weight
exc_s.exc_w = exc_w_0


# Synapses from inh to exc **************************************************
inh_s = Synapses(
    inh_layer,
    exc_layer,
    inh_dynamic,
    on_pre = inh_pre_action
)

inh_s.connect(condition = "i != j")

# initialize weight
inh_s.inh_w = inh_w_0

# Set up data recording ======================================================

input_spike = SpikeMonitor(input_layer)
exc_spike = SpikeMonitor(exc_layer)
inh_spike = SpikeMonitor(inh_layer)

# pre_update_t = StateMonitor(input_s, "pre_last", record=True)

# input_w = StateMonitor(input_s, "w", record=True)

# delta_w_mo = StateMonitor(input_s, "delta_w", record=True)

# exc_v = StateMonitor(exc_layer, "v", record=True) 

# exc_vt = StateMonitor(exc_layer, "vt_exc", record=True)


#inh_state = StateMonitor(inh_layer, 'v', record = True)

# run(50*ms)

# exc_state = StateMonitor(exc_layer, 'v', record = True)
# neuron_vt_record = SpikeMonitor(exc_layer)

weights_2d_tot = []
updates_2d_tot = []
threshold_2d_tot = []

In [86]:
# read in neuron class label
neuron_class = []
with open("./neuron_class.txt", "r") as f:
    n_neuron = int(next(f))
    for line in f:
        neuron_class.append(int(line))

In [87]:
# prepare variable vector

neuron_activity = np.zeros(100)

pred_label = []

In [88]:
# feature-training model

step_size = 20

start = 0

stop = start + step_size

for sample in range(start, stop):    
    # sequantially lower certain neuron's threshold
    input_s

    # print("{0:.2g}% done...".format(sample / n_tot_sample * 100))

    # set input layer rates
    input_layer.rates = data_test[sample] * rate_multiplier

    # learning rate decay after first 100 samples
    #if sample > 100:
    #    eta = eta / sqrt(sample - 100 )

    # reset last pre-synaptic neuron fire time to 0
    #input_s.pre_last = 0*ms

    # reset voltage to 0
    exc_layer.v = 0

    # reset fire time
    exc_layer.n_fire = 0

    # reset total_offset
    #input_s.total_offset = 0

    # reset neuron activity
    neuron_activity[:] = 0
    
    run(100*ms)

    for exc_index in range(100):
            neuron_activity[exc_index] = exc_layer.n_fire[exc_index]
    
    pred_label.append(neuron_class[np.argmax(neuron_activity)])

    if sample % 1000 == 0:
        print("{0}% completed".format(100 * sample / 10000))



0.0% completed


In [38]:
int(100 * sample / 10000)

0

In [32]:
print("{0}% completed".format(100 * sample / 10000))

0.04% completed


In [28]:
pred_label

[7, 2, 1, 0, 4]

In [59]:
tot_per_number = np.zeros(10)
correct_per_number = np.zeros(10)
accuracy_per_number = np.zeros(10)

In [60]:
correct_flag = np.zeros(10000)

for sample in range(10000):
    tot_per_number[label_test[sample]] = tot_per_number[label_test[sample]] + 1
    if pred_label[sample] == label_test[sample]:
        correct_flag[sample] = 1
        correct_per_number[label_test[sample]] = correct_per_number[label_test[sample]]  + 1
        
accuracy_per_number = correct_per_number / tot_per_number

In [53]:
accuracy = np.sum(correct_flag) / 10000

In [55]:
accuracy

0.7684

In [62]:
for n in range(10):
    print("{0} : {1:.2f}%".format(n, accuracy_per_number[n] * 100))

0 : 90.71%
1 : 91.98%
2 : 71.71%
3 : 84.16%
4 : 52.85%
5 : 70.40%
6 : 95.09%
7 : 78.02%
8 : 58.11%
9 : 72.84%


In [63]:
np.std(accuracy_per_number)

0.13488115816816637

In [89]:
len(input_spike.i)

93232

In [90]:
len(input_spike.t)

93232

In [96]:
input_spike.t[10000] / ms

235.3

In [136]:
time_slot = 2000

input_spike_entry = []

for n in range(784):
    input_spike_entry.append([])

for n in range(len(input_spike.i)):
    if input_spike.t[n] / ms >= time_slot and input_spike.t[n] / ms < time_slot + 100:
        input_spike_entry[input_spike.i[n]].append(input_spike.t[n])

with open("./model 0.3/spike/input/input_{0}.txt".format(time_slot), "w") as f:
    f.write("{0}\n".format(784))
    for n in range(784):
        for m in input_spike_entry[n]:
            f.write("{0:.1f} ".format(m / ms))
        f.write("\n")

In [159]:
time_slot = 2000

exc_spike_entry = []

for n in range(100):
    exc_spike_entry.append([])

for n in range(len(exc_spike.i)):
    if exc_spike.t[n] / ms >= time_slot and exc_spike.t[n] / ms < time_slot + 100:
        exc_spike_entry[exc_spike.i[n]].append(exc_spike.t[n])

with open("./model 0.3/spike/exc/exc_{0}.txt".format(time_slot), "w") as f:
    f.write("{0}\n".format(100))
    for n in range(100):
        for m in exc_spike_entry[n]:
            f.write("{0:.1f} ".format(m / ms))
        f.write("\n")

In [160]:
for time in range(20):

    time_slot = time * 100

    inh_spike_entry = []

    for n in range(100):
        inh_spike_entry.append([])

    for n in range(len(inh_spike.i)):
        if inh_spike.t[n] / ms >= time_slot and inh_spike.t[n] / ms < time_slot + 100:
            inh_spike_entry[inh_spike.i[n]].append(inh_spike.t[n])

    with open("./model 0.3/spike/inh/inh_{0}.txt".format(time_slot), "w") as f:
        f.write("{0}\n".format(100))
        for n in range(100):
            for m in inh_spike_entry[n]:
                f.write("{0:.1f} ".format(m / ms))
            f.write("\n")